In [1]:
# 基本パッケージ（numpy,Pandas,matplotlib）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# サポートベクターマシーン
from sklearn import svm
# train_test_split（データを分割出してくれる）
from sklearn.model_selection import train_test_split
# accuracy_score（正解率を測れる）
from sklearn.metrics import accuracy_score
# グリッドサーチ（ハイパーパラメータを自動的に最適化してくれる）
from sklearn.model_selection import GridSearchCV
# 正規化、標準化用
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# 特徴量選択用
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

In [2]:
# 株価データの読み込み
stock_data = pd.read_csv("stock_price.csv", encoding="shift-jis")
print(stock_data)

              日付     始値     高値     安値     終値      出来高  終値調整値
0     2001-07-13  12490  12490  12300  12310   259430  12310
1     2001-07-16  12400  12400  12250  12330    99740  12330
2     2001-07-17  12170  12170  12100  12120    93150  12120
3     2001-07-18  12150  12150  11830  11840   165330  11840
4     2001-07-19  11990  11990  11860  11920   620220  11920
5     2001-07-23  11950  11950  11540  11570   170700  11570
6     2001-07-24  11560  11860  11560  11840   173370  11840
7     2001-07-25  11750  12020  11750  11860   207180  11860
8     2001-07-26  11940  11960  11810  11830   396780  11830
9     2001-07-27  11790  11940  11690  11820  1801620  11820
10    2001-07-30  11900  11900  11540  11580   311850  11580
11    2001-07-31  11670  11870  11660  11850   183190  11850
12    2001-08-01  11950  12000  11810  12000   310730  12000
13    2001-08-02  12120  12420  12120  12390   640190  12390
14    2001-08-03  12360  12360  12230  12240   299960  12240
15    2001-08-06  12160 

In [3]:
# 要素数の設定
count_s = len(stock_data)

In [4]:
# 各要素を上昇率へ変換する
modified_data = np.zeros((0,6))
for i in range(1,count_s):
    modified_data = np.append(
        modified_data, 
        np.array([[float(stock_data.loc[i,['始値']] - stock_data.loc[i-1,['始値']])/float(stock_data.loc[i-1,['始値']]),
                   float(stock_data.loc[i,['高値']] - stock_data.loc[i-1,['高値']])/float(stock_data.loc[i-1,['高値']]),
                   float(stock_data.loc[i,['安値']] - stock_data.loc[i-1,['安値']])/float(stock_data.loc[i-1,['安値']]),
                   float(stock_data.loc[i,['終値']] - stock_data.loc[i-1,['終値']])/float(stock_data.loc[i-1,['終値']]),
                   float(stock_data.loc[i,['出来高']] - stock_data.loc[i-1,['出来高']])/float(stock_data.loc[i-1,['出来高']]),
                   float(stock_data.loc[i,['終値調整値']] - stock_data.loc[i-1,['終値調整値']])/float(stock_data.loc[i-1,['終値調整値']])]]),
        axis = 0)

In [5]:
# 要素数の設定
count_m = len(modified_data)

# 最終日のデータを削除
successive_data = np.delete(modified_data ,count_m - 1, axis=0)

# 正解値を格納するリスト　価格上昇: 1 価格低下:0
answers = []

# 正解値の格納
for i in range(1, count_m):
    # 上昇率が0以上なら1、そうでないなら0を格納
    if modified_data[i,3] > 0:
        answers.append(1)
    else:
        answers.append(0)
 

In [6]:
# データの分割（データの80%を訓練用に、20％をテスト用に分割する）
X_train, X_test, y_train, y_test = train_test_split(successive_data, answers, test_size=0.2, random_state=1,shuffle=False)

In [7]:
# グリッドサーチするパラメータを設定
parameters = {'C':[1, 3, 5],'loss':('hinge', 'squared_hinge')}
 
# グリッドサーチを実行
clf = GridSearchCV(svm.LinearSVC(), parameters)
clf.fit(X_train, y_train) 
 
# グリッドサーチ結果(最適パラメータ)を取得
GS_C, GS_loss = clf.best_params_.values()
print ("最適パラメータ：{}".format(clf.best_params_))

最適パラメータ：{'C': 1, 'loss': 'squared_hinge'}


In [8]:
# 最適パラメーターを指定して学習
clf = svm.LinearSVC(loss=GS_loss, C=GS_C,random_state=1)
clf.fit(X_train , y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=1, tol=0.0001,
     verbose=0)

In [9]:
# 学習後のモデルによるテスト
# トレーニングデータを用いた予測
y_train_pred = clf.predict(X_train)
# テストデータを用いた予測
y_val_pred = clf.predict(X_test)

In [10]:
# 正解率の計算
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_val_pred)
# 正解率を表示
print("トレーニングデータに対する正解率：" + str(train_score * 100) + "%")
print("テストデータに対する正解率：" + str(test_score * 100) + "%")

トレーニングデータに対する正解率：51.76224611708483%
テストデータに対する正解率：48.21002386634845%
